## Constructing a Minimum-Variance Portfolio

First, we need to import all of the libraries that we'll be using:

* NumPy
* pandas
* yfinance
* optimize from scipy

In [278]:
# Imports
import numpy as np
import pandas as pd
import yfinance as yf
from scipy import optimize

### Collecting and Preparing the Data

In [279]:
tickers = [
    'AAPL',
    'MSFT',
    'GOOG',
    'AMZN'
]

period = '1y'

interval = '1d'

fields = []
for i in range(len(tickers)):
    fields.append((tickers[i], 'Close'))

data = yf.download(
    tickers = tickers,
    period = period,
    interval = interval,
    group_by = 'ticker'
)[fields]

print(data.head())

[*********************100%***********************]  4 of 4 completed
                  AAPL        MSFT        GOOG        AMZN
                 Close       Close       Close       Close
Date                                                      
2022-01-19  166.229996  303.329987  135.651993  156.298996
2022-01-20  164.509995  301.600006  133.506500  151.667496
2022-01-21  162.410004  296.029999  130.091995  142.643005
2022-01-24  161.619995  296.369995  130.371994  144.544006
2022-01-25  159.779999  288.489990  126.735497  139.985992


Returns

In [280]:
returns = data.pct_change().dropna() * 100


print(returns.head())

                AAPL      MSFT      GOOG      AMZN
               Close     Close     Close     Close
Date                                              
2022-01-20 -1.034712 -0.570330 -1.581615 -2.963231
2022-01-21 -1.276513 -1.846819 -2.557557 -5.950181
2022-01-24 -0.486429  0.114852  0.215231  1.332698
2022-01-25 -1.138471 -2.658840 -2.789324 -3.153375
2022-01-26 -0.056325  2.849319  1.976170 -0.795433


Covariance Matrix

In [281]:
covariance = returns.cov().to_numpy()

print(covariance)

[[ 5.09994322  4.08001946  4.30287533  4.93906319]
 [ 4.08001946  4.97729435  4.58205068  5.27923042]
 [ 4.30287533  4.58205068  5.95079833  5.69039874]
 [ 4.93906319  5.27923042  5.69039874 10.22353071]]


### Optimization

Objective Function and constraint

In [282]:
def portfolioVar(w, cov):
    w = np.matrix(w)
    cov = np.matrix(cov)
    result = w * cov * w.T
    return result

constraint = ({'type': 'eq', 'fun': lambda x:  np.sum(x) - 1})

bounds = tuple((-1, 1) for i in tickers)

((-1, 1), (-1, 1), (-1, 1), (-1, 1))


Initial guess, bounds, and optimal portfolio

In [283]:

w0 = [1 / len(tickers)] * len(tickers)


result = optimize.minimize(
    fun = portfolioVar,
    x0 = w0,
    args = covariance,
    method = 'SLSQP',
    bounds = bounds,
    constraints = constraint
)

result.x = result.x.round(3)

for i in range(len(tickers)):
    print(tickers[i], '=', result.x[i])

AAPL = 0.469
MSFT = 0.549
GOOG = 0.125
AMZN = -0.143
